In [40]:
pip install openai PyPDF2


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [41]:
# file: interview_coach.py
import os
import sys
from getpass import getpass
from datetime import datetime
from typing import Optional

try:
    from openai import OpenAI
except Exception as e:
    print("OpenAI SDK not found. Install with: pip install openai")
    raise

# ---------- Helpers ----------
def read_resume_text() -> str:
    """
    Let user choose: paste resume text OR load from a file (.txt or .pdf).
    """
    choice = input("Resume input — (1) paste text  (2) load from file? [1/2]: ").strip() or "1"
    if choice == "1":
        print("\nPaste your resume text below. End input with an empty line:")
        lines = []
        while True:
            try:
                line = input()
            except EOFError:
                break
            if line.strip() == "":
                # double-enter to end; but allow single empty lines inside content
                confirm = input("Finish? (y/n): ").strip().lower()
                if confirm == "y":
                    break
                else:
                    continue
            lines.append(line)
        return "\n".join(lines).strip()

    path = input("Enter resume file path (.txt or .pdf): ").strip().strip('"').strip("'")
    if not os.path.exists(path):
        print("File not found.")
        sys.exit(1)
    if path.lower().endswith(".txt") or path.lower().endswith(".md"):
        with open(path, "r", encoding="utf-8", errors="ignore") as f:
            return f.read()
    elif path.lower().endswith(".pdf"):
        try:
            import PyPDF2
        except Exception:
            print("PyPDF2 not installed. Run: pip install PyPDF2")
            sys.exit(1)
        text = []
        with open(path, "rb") as f:
            reader = PyPDF2.PdfReader(f)
            for page in reader.pages:
                try:
                    text.append(page.extract_text() or "")
                except Exception:
                    pass
        return "\n".join(text).strip()
    else:
        print("Unsupported file type. Use .txt or .pdf")
        sys.exit(1)

def ensure_openai_client(existing_key: Optional[str] = None) -> OpenAI:
    key = existing_key or os.environ.get("OPENAI_API_KEY")
    if not key:
        # hidden prompt
        key = getpass("Paste your OpenAI API key (input hidden): ").strip()
    if not key.startswith("sk-"):
        print("That doesn't look like a valid API key.")
        sys.exit(1)
    os.environ["OPENAI_API_KEY"] = key
    return OpenAI()

# ---------- Prompt Builder ----------
SYSTEM_INSTRUCTIONS = """You are an expert interview coach.
Create realistic interview questions and tailored model answers that reference the candidate's resume.
Prefer concrete, resume-grounded specifics over generic advice.
When information is missing, suggest how the candidate can bridge the gap truthfully."""

USER_TEMPLATE = """JOB ROLE:
{job_role}

RESUME (verbatim):
{resume}

TASKS:
1) Extract a concise candidate profile:
   - Core skills & tech stack
   - Domains/industries
   - Notable projects & quantified impact (if any)
   - Leadership/collaboration highlights

2) Generate likely interview questions (18–24 total), balanced across:
   - Role-specific / technical
   - Behavioral (STAR)
   - System/architecture or case-style (if relevant to the role)
   - Culture/values & collaboration
   - Questions THEY should ask the interviewer

3) For each question, provide:
   - A strong model answer tailored to THIS resume (5–9 sentences)
   - 3 talk-tracks/bullets the candidate can riff on
   - If resume lacks direct experience, give an honest “experience-bridge” answer using adjacent skills

4) Summarize 6–10 rapid-fire one-liners (short answers) that the candidate can memorize.

5) Finish with a practice plan (7 days), including daily drills and mock prompts.

FORMAT STRICTLY AS MARKDOWN with clear headings:
# Candidate Snapshot
# Likely Interview Questions & Model Answers
## [Category] Q1: ...
- Model Answer:
- Talk Tracks:
- Experience Bridge (if needed):

# Rapid-Fire One-Liners
# Questions to Ask the Interviewer
# 7-Day Practice Plan
"""

def build_prompt(job_role: str, resume: str) -> str:
    # Trim overly long resumes to avoid context overflow
    max_chars = 18000
    trimmed = resume if len(resume) <= max_chars else resume[:max_chars] + "\n...[trimmed]"
    return USER_TEMPLATE.format(job_role=job_role.strip(), resume=trimmed.strip())

# ---------- Core ----------
def generate_coaching_pack(client: OpenAI, job_role: str, resume_text: str, model: str = "gpt-4o-mini") -> str:
    user_prompt = build_prompt(job_role, resume_text)
    resp = client.chat.completions.create(
        model=model,
        temperature=0.4,
        messages=[
            {"role": "system", "content": SYSTEM_INSTRUCTIONS},
            {"role": "user", "content": user_prompt},
        ],
    )
    return resp.choices[0].message.content

def save_markdown(content: str, job_role: str) -> str:
    safe_role = "".join(c for c in job_role if c.isalnum() or c in ("-", "_", " ")).strip().replace(" ", "_")
    out_dir = "output"
    os.makedirs(out_dir, exist_ok=True)
    fname = f"Interview_Coach_{safe_role}_{datetime.now().strftime('%Y%m%d_%H%M')}.md"
    path = os.path.join(out_dir, fname)
    with open(path, "w", encoding="utf-8") as f:
        f.write(content)
    return path

# ---------- Main ----------
def main():
    print("=== Interview Q&A Coach ===")
    job_role = input("Target job role (e.g., 'Data Analyst at fintech', 'SDE-1 Backend'): ").strip()
    if not job_role:
        print("Job role is required.")
        return
    resume_text = read_resume_text()
    if not resume_text:
        print("No resume content provided.")
        return

    client = ensure_openai_client()
    print("\nGenerating your tailored Q&A pack... ⏳")
    try:
        md = generate_coaching_pack(client, job_role, resume_text)
    except Exception as e:
        print(f"OpenAI error: {e}")
        return

    out_path = save_markdown(md, job_role)
    print(f"\n✅ Done! Saved to: {out_path}")
    print("Open the Markdown file for a clean, study-ready layout.")

if __name__ == "__main__":
    main()


=== Interview Q&A Coach ===

Generating your tailored Q&A pack... ⏳

✅ Done! Saved to: output\Interview_Coach_SDE-1_20250809_2302.md
Open the Markdown file for a clean, study-ready layout.
